In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pylab
import seaborn as sns; sns.set(color_codes=True)
from matplotlib.colors import ListedColormap
import networkx as nx

In [20]:
region = pd.read_pickle('wrds_geo_exp.pkl')

In [21]:
region.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16724649 entries, 0 to 16724648
Data columns (total 4 columns):
company_id      object
cusip           object
name            object
layer_number    float64
dtypes: float64(1), object(3)
memory usage: 510.4+ MB


In [22]:
region.head()

,company_id,cusip,name,layer_number
0,10001,879080109,General / Global,0.0
1,10001,879080109,Americas,1.0
2,10001,879080109,North America,2.0
3,10001,879080109,United States and Canada,3.0
4,10001,879080109,United States,4.0


In [23]:
region_0 = region.loc[region['layer_number'] == 0]
region_1 = region.loc[region['layer_number'] == 1]
region_2 = region.loc[region['layer_number'] == 2]
region_3 = region.loc[region['layer_number'] == 3]
region_4 = region.loc[region['layer_number'] == 4]

In [24]:
print(len(region_0['name'].unique()),
len(region_1['name'].unique()), 
len(region_2['name'].unique()),
len(region_3['name'].unique()),
len(region_4['name'].unique()))

1 7 15 30 258


In [25]:
len(region_3['cusip'].unique())

53650

In [26]:
region_3 = region_3.drop_duplicates(subset=['cusip'], keep='last')

In [27]:
region_3 = region_3.rename(columns = {'cusip': 'cusip_9'})

In [28]:
region_3.head()

,company_id,cusip_9,name,layer_number
73,10001,879080109,United States and Canada,3.0
404,10004,878919208,Western Middle East,3.0
529,100050538,A1142C109,Europe Unallocated Area,3.0
534,100050999,Y8588E105,Far East,3.0
539,100051504,74737F101,Non-Operating Area,3.0


In [29]:
region_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53650 entries, 73 to 16724646
Data columns (total 4 columns):
company_id      53650 non-null object
cusip_9         53649 non-null object
name            53650 non-null object
layer_number    53650 non-null float64
dtypes: float64(1), object(3)
memory usage: 2.0+ MB


In [30]:
region_3.to_pickle('region_3.pkl')

In [31]:
net_degree = pd.read_pickle('net_degree.pkl')

In [32]:
net_degree.head()

,cusip_9,out_degree_centrality,in_degree_centrality,closeness_centrality,betweenness_centrality,eigenvector_centrality,harmonic_centrality,average_neighbor_degree,square_clustering,core_number,...,out_degree_0,in_degree_1,out_degree_1,in_diff,out_diff,out_diff_ratio,in_diff_ratio,higher_0,lower_0,out_group
0,000360206,0.000846,0.000000,0.000000,0.000000,7.397798e-18,0.000000,1.666667,0.0,3.0,...,3.0,0.0,3.0,0.0,0.0,0.0,NaN,NaN,NaN,3.0
1,000361105,0.000564,0.000282,0.140794,0.000022,6.768081e-03,556.132179,5.000000,0.0,3.0,...,2.0,0.0,5.0,-1.0,3.0,1.5,-1.000000,1.5,NaN,1.0
2,00089L107,0.000282,0.000000,0.000000,0.000000,7.397798e-18,0.000000,0.000000,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,3.0
3,001031103,0.000282,0.000000,0.000000,0.000000,7.397798e-18,0.000000,1.000000,0.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,3.0
4,001084102,0.000564,0.002538,0.122048,0.001122,1.466880e-03,476.103355,6.000000,0.0,6.0,...,2.0,7.0,2.0,-2.0,0.0,0.0,-0.222222,NaN,NaN,3.0


In [33]:
net_degree.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83041 entries, 0 to 15257
Data columns (total 25 columns):
cusip_9                    83041 non-null object
out_degree_centrality      83041 non-null float64
in_degree_centrality       83041 non-null float64
closeness_centrality       83041 non-null float64
betweenness_centrality     83041 non-null float64
eigenvector_centrality     83041 non-null float64
harmonic_centrality        83041 non-null float64
average_neighbor_degree    83041 non-null float64
square_clustering          83041 non-null float64
core_number                83041 non-null float64
pagerank                   83041 non-null float64
hubs                       83041 non-null float64
authorities                83041 non-null float64
year_0                     83041 non-null object
in_degree_0                83041 non-null float64
out_degree_0               83041 non-null float64
in_degree_1                83041 non-null float64
out_degree_1               83041 non-null f

In [34]:
net_degree_region = pd.merge(net_degree, region_3, how = 'inner', on = 'cusip_9')

In [35]:
net_degree_region = net_degree_region.drop(columns = ['company_id','layer_number'])

In [36]:
net_degree_region = net_degree_region.rename(columns = {'name':'region'})

In [37]:
region_dummy = pd.get_dummies(net_degree_region['region'])

In [38]:
net_degree_region = pd.concat([net_degree_region,region_dummy], axis = 1)

In [39]:
net_degree_region.to_pickle('net_degree_region.pkl')

In [42]:
train_2010 = net_degree_region.loc[net_degree_region['year_0']< '2011-12-31']
test_2011 = net_degree_region.loc[net_degree_region['year_0'] == '2011-12-31']

In [43]:
train_2010.to_pickle('train_2010.pkl')
test_2011.to_pickle('test_2011.pkl')